In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np
import itertools

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
# Each ball can be either -1 or +1
options = [-1, 1]

# Generate all possible configurations
all_configurations = list(itertools.product(options, repeat=16))
print (type(all_configurations[1]))
print (list(all_configurations[1]))

<class 'tuple'>
[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1]


In [4]:
def right_side(b, b1, b2, b3, b4, w15, w16, w116, w11, w27, w28, w29, w22, w310, w311, w312, w33, w413, w414, w415, w44, w1, w2, w3, w4):
    x = all_configurations[1]
    m = torch.tensor(0 + 1j) * torch.pi / 4
#     for i in range(len(x)):
    return (torch.cosh(torch.tensor(b1 + w15*x[4] + w16*x[5] + w116*x[15] - w11*x[0]))*
               torch.cosh(torch.tensor(b2 + w27*x[6] + w28*x[7] + w29*x[8] - w22*x[1]))*
                torch.cosh(torch.tensor(b3 + w310*x[9] + w311*x[10] + w312*x[11] - w33*x[2]))*
                torch.cosh(torch.tensor(b4 + w413*x[12] + w414*x[13] + w415*x[14] - w44*x[3]))*
                torch.cosh(torch.tensor(b - w1*x[0] - w2*x[1] - w3*x[2] - w4*x[3]))* 
                torch.cosh(m*torch.tensor(x[14] + x[15] - x[0] - x[3]))*torch.cosh(m*torch.tensor(x[6] + x[7] - x[0] - x[1]))*
                torch.cosh(m*torch.tensor(x[11] + x[12] - x[2] - x[3]))* torch.cosh(m*torch.tensor(x[8] + x[9] - x[1] - x[2]))
)

In [4]:
a = right_side(0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0)
a

tensor(-4.3711e-08+0.j)

In [5]:
def left_side(b, b1, b2, b3, b4, w15, w16, w116, w11, w27, w28, w29, w22, w310, w311, w312, w33, w413, w414, w415, w44, w1, w2, w3, w4):
    x = all_configurations[1]
    m = torch.tensor(0 + 1j) * torch.pi / 4
    for i in range(len(x)):
        return (torch.cosh(torch.tensor(b1 + w15*x[4] + w16*x[5] + w116*x[15] + w11*x[0]))*
               torch.cosh(torch.tensor(b2 + w27*x[6] + w28*x[7] + w29*x[8] + w22*x[1]))*
                torch.cosh(torch.tensor(b3 + w310*x[9] + w311*x[10] + w312*x[11] + w33*x[2]))*
                torch.cosh(torch.tensor(b4 + w413*x[12] + w414*x[13] + w415*x[14] + w44*x[3]))*
                torch.cosh(torch.tensor(b + w1*x[0] + w2*x[1] + w3*x[2] + w4*x[3]))* 
                torch.cosh(m*torch.tensor(x[14] + x[15] + x[0] + x[3]))*torch.cosh(m*torch.tensor(x[6] + x[7] + x[0] + x[1]))*
                torch.cosh(m*torch.tensor(x[11] + x[12] + x[2] + x[3]))* torch.cosh(m*torch.tensor(x[8] + x[9] + x[1] + x[2]))
)

In [6]:
b = left_side(0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0)
b

tensor(4.3711e-08+0.j)

In [5]:
def opt_func(b, b1, b2, b3, b4, w15, w16, w116, w11, w27, w28, w29, w22, w310, w311, w312, w33, w413, w414, w415, w44, w1, w2, w3, w4):
    m = torch.tensor(0 + 1j) * torch.pi / 4
    right_side = []
    left_side = []
    # Each spin can be either -1 or +1
    options = [-1, 1]

    # Generate all possible configurations
    all_configurations = list(itertools.product(options, repeat=16))
    
    # Write each side of the equations based on the configurations
    for i in range(len(all_configurations)):
        conf = all_configurations[i]
        # Write the right hand side of each equation
        right_side.append(torch.cosh(torch.tensor(b1 + w15*conf[4] + w16*conf[5] + w116*conf[15] - w11*conf[0]))*
               torch.cosh(torch.tensor(b2 + w27*conf[6] + w28*conf[7] + w29*conf[8] - w22*conf[1]))*
                torch.cosh(torch.tensor(b3 + w310*conf[9] + w311*conf[10] + w312*conf[11] - w33*conf[2]))*
                torch.cosh(torch.tensor(b4 + w413*conf[12] + w414*conf[13] + w415*conf[14] - w44*conf[3]))*
                torch.cosh(torch.tensor(b - w1*conf[0] - w2*conf[1] - w3*conf[2] - w4*conf[3]))* 
                torch.cosh(m*torch.tensor(conf[14] + conf[15] - conf[0] - conf[3]))*
                  torch.cosh(m*torch.tensor(conf[6] + conf[7] - conf[0] - conf[1]))*
                torch.cosh(m*torch.tensor(conf[11] + conf[12] - conf[2] - conf[3]))* 
                  torch.cosh(m*torch.tensor(conf[8] + conf[9] - conf[1] - conf[2])))
    
        # Write the left hand side of each equation
        left_side.append(torch.cosh(torch.tensor(b1 + w15*conf[4] + w16*conf[5] + w116*conf[15] + w11*conf[0]))*
               torch.cosh(torch.tensor(b2 + w27*conf[6] + w28*conf[7] + w29*conf[8] + w22*conf[1]))*
                torch.cosh(torch.tensor(b3 + w310*conf[9] + w311*conf[10] + w312*conf[11] + w33*conf[2]))*
                torch.cosh(torch.tensor(b4 + w413*conf[12] + w414*conf[13] + w415*conf[14] + w44*conf[3]))*
                torch.cosh(torch.tensor(b + w1*conf[0] + w2*conf[1] + w3*conf[2] + w4*conf[3]))* 
                torch.cosh(m*torch.tensor(conf[14] + conf[15] + conf[0] + conf[3]))*
                  torch.cosh(m*torch.tensor(conf[6] + conf[7] + conf[0] + conf[1]))*
                torch.cosh(m*torch.tensor(conf[11] + conf[12] + conf[2] + conf[3]))* 
                  torch.cosh(m*torch.tensor(conf[8] + conf[9] + conf[1] + conf[2])))
    #print (len(right_side), len(left_side))
    # Write the optimization function
    subs = [(a - b)**2 for a, b in zip(left_side, right_side)]
    loss = sum(subs)
    
    return torch.abs(loss)

In [2]:
def non_linear(b, b1, b2, b3, b4, w15, w16, w116, w11, w27, w28, w29, w22, w310, w311, w312, w33, w413, w414, w415, w44, w1, w2, w3, w4):
    
    # initialization
    m = torch.tensor(complex(0, torch.pi/4))
    right_side = []
    left_side = []
    
    # Each spin can be either -1 or +1
    options = [-1, 1]

    # Generate all possible configurations
    all_configurations = list(itertools.product(options, repeat=16))
    
    # Write each side of the equations based on the configurations
    for i in range(len(all_configurations)):
        conf = all_configurations[i]
        conf_tensor = torch.tensor(conf, dtype=torch.float32)
        
        # creating complex tensors
        z1 = torch.complex(b1, w15*conf_tensor[4] + w16*conf_tensor[5] + w116*conf_tensor[15] - w11*conf_tensor[0])
        z2 = torch.complex(b2, w27*conf_tensor[6] + w28*conf_tensor[7] + w29*conf_tensor[8] - w22*conf_tensor[1])
        z3 = torch.complex(b3, w310*conf_tensor[9] + w311*conf_tensor[10] + w312*conf_tensor[11] - w33*conf_tensor[2])
        z4 = torch.complex(b4, w413*conf_tensor[12] + w414*conf_tensor[13] + w415*conf_tensor[14] - w44*conf_tensor[3])
        z = torch.complex(b, - w1*conf_tensor[0] - w2*conf_tensor[1] - w3*conf_tensor[2] - w4*conf_tensor[3])

        a1 = torch.complex(b1, w15*conf_tensor[4] + w16*conf_tensor[5] + w116*conf_tensor[15] + w11*conf_tensor[0])
        a2 = torch.complex(b2, w27*conf_tensor[6] + w28*conf_tensor[7] + w29*conf_tensor[8] + w22*conf_tensor[1])
        a3 = torch.complex(b3, w310*conf_tensor[9] + w311*conf_tensor[10] + w312*conf_tensor[11] + w33*conf_tensor[2])
        a4 = torch.complex(b4, w413*conf_tensor[12] + w414*conf_tensor[13] + w415*conf_tensor[14] + w44*conf_tensor[3])
        a = torch.complex(b, + w1*conf_tensor[0] + w2*conf_tensor[1] + w3*conf_tensor[2] + w4*conf_tensor[3])

        # Write the right hand side of each equation
        right_side.append(torch.cosh(a)*torch.cosh(a1)*torch.cosh(a2)*torch.cosh(a3)*torch.cosh(a4)*
                         torch.cosh(m*torch.complex(conf_tensor[14] + conf_tensor[15] - conf_tensor[0] - conf_tensor[3], torch.tensor(0.0)))*
                         torch.cosh(m*torch.complex(conf_tensor[6] + conf_tensor[7] - conf_tensor[0] - conf_tensor[1], torch.tensor(0.0)))*
                         torch.cosh(m*torch.complex(conf_tensor[11] + conf_tensor[12] - conf_tensor[2] - conf_tensor[3], torch.tensor(0.0)))*
                         torch.cosh(m*torch.complex(conf_tensor[8] + conf_tensor[9] - conf_tensor[1] - conf_tensor[2], torch.tensor(0.0))) 
                         )
        
        # write teh left hand side of each equation
        left_side.append(torch.cosh(z)*torch.cosh(z1)*torch.cosh(z2)*torch.cosh(z3)*torch.cosh(z4)*
                         torch.cosh(m*torch.complex(conf_tensor[14] + conf_tensor[15] - conf_tensor[0] - conf_tensor[3], torch.tensor(0.0)))*
                         torch.cosh(m*torch.complex(conf_tensor[6] + conf_tensor[7] - conf_tensor[0] - conf_tensor[1], torch.tensor(0.0)))*
                         torch.cosh(m*torch.complex(conf_tensor[11] + conf_tensor[12] - conf_tensor[2] - conf_tensor[3], torch.tensor(0.0)))*
                         torch.cosh(m*torch.complex(conf_tensor[8] + conf_tensor[9] - conf_tensor[1] - conf_tensor[2], torch.tensor(0.0))) 
                         )
    
    # Write the optimization function
    subs = [(a - b)**2 for a, b in zip(left_side, right_side)]
    loss = sum(subs)
    
    return torch.abs(loss)

In [ ]:
# Neural Network architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(25, 100)
        self.fc2 = nn.Linear(100, 25)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        return self.fc2(x)

# Initialize the neural network
net = Net()

# Generate training data
x = torch.rand(1200, 25)

# Split data into training and validation sets
x_train, x_val = train_test_split(x.numpy(), test_size=0.2, random_state=42)
x_train, x_val = torch.tensor(x_train, dtype=torch.float32), torch.tensor(x_val, dtype=torch.float32)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=0.01)

# Lists to store loss values
train_losses = []
val_losses = []

# Training loop
for epoch in range(1000):
    optimizer.zero_grad()

    # Forward pass with training data
    inputs_train = net(x_train)
    outputs_train = non_linear(inputs_train[:, 0], inputs_train[:, 1], inputs_train[:, 2], inputs_train[:, 3], inputs_train[:, 4], inputs_train[:, 5], inputs_train[:, 6],
                             inputs_train[:, 7], inputs_train[:, 8], inputs_train[:, 9], inputs_train[:, 10], inputs_train[:, 11], inputs_train[:, 12], inputs_train[:, 13],
                             inputs_train[:, 14], inputs_train[:, 15], inputs_train[:, 16], inputs_train[:, 17], inputs_train[:, 18], inputs_train[:, 18], inputs_train[:, 20],
                             inputs_train[:, 21], inputs_train[:, 22], inputs_train[:, 23], inputs_train[:, 24])
    loss_train = criterion(outputs_train, torch.zeros(outputs_train.size(), dtype=torch.float32))
    loss_train.backward()
    optimizer.step()

    # Validation
    with torch.no_grad():
        inputs_val = net(x_val)
        outputs_val = non_linear(inputs_val[:, 0], inputs_val[:, 1], inputs_val[:, 2], inputs_val[:, 3], inputs_val[:, 4], inputs_val[:, 5], inputs_val[:, 6],
                               inputs_val[:, 7], inputs_val[:, 8], inputs_val[:, 9], inputs_val[:, 10], inputs_val[:, 11], inputs_val[:, 12], inputs_val[:, 13],
                               inputs_val[:, 14], inputs_val[:, 15], inputs_val[:, 16], inputs_val[:, 17], inputs_val[:, 18], inputs_val[:, 18], inputs_val[:, 20],
                               inputs_val[:, 21], inputs_val[:, 22], inputs_val[:, 23], inputs_val[:, 24])
        loss_val = criterion(outputs_val, torch.zeros(outputs_val.size(), dtype=torch.float32))

    train_losses.append(loss_train.item())
    val_losses.append(loss_val.item())

    print(f'Epoch {epoch}, Training Loss: {loss_train.item()}, Validation Loss: {loss_val.item()}')

# Plot the training and validation loss
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [31]:
type(inputs_train)

torch.Tensor

In [14]:
# Predict the set of inputs that would minimize the function
with torch.no_grad():
    predicted_inputs = net(torch.zeros(1, 25))
    optimal_values = predicted_inputs.numpy()[0]

    # Scale y1, y2, y3, y4 to be in terms of multiples of pi
    y1_scaled = optimal_values[1] / torch.pi
    y2_scaled = optimal_values[3] / torch.pi
    y3_scaled = optimal_values[5] / torch.pi
    y4_scaled = optimal_values[6] / torch.pi

    minimized_output = complex_function(predicted_inputs[0, 0], predicted_inputs[0, 1], predicted_inputs[0, 2], predicted_inputs[0, 3], predicted_inputs[0, 4], predicted_inputs[0, 5], predicted_inputs[0, 6])

print(f'Optimal values for minimizing the function: {optimal_values}')
print(f'y1, y2, y3, y4 in terms of multiples of π: {y1_scaled}πi, {y2_scaled}πi, {y3_scaled}πi, {y4_scaled}πi')
print(f'Minimized function output: {minimized_output}')

Optimal values for minimizing the function: [ 0.01971548  0.83678234  0.6873839  -0.06972719 -0.03705344 -0.8869992
  0.8952646 ]
y1, y2, y3, y4 in terms of multiples of π: 0.2663560902075841πi, -0.02219485386153145πi, -0.2823406111674357πi, 0.2849715810629769πi
Minimized function output: 0.2280472368001938


In [28]:
torch.complex(torch.tensor(0.0),torch.tensor(3.0))

tensor(0.+3.j)

In [25]:
x = np.random.rand(1200, 25)
x

array([[0.86083464, 0.86580223, 0.47939576, ..., 0.47617179, 0.55283667,
        0.22405356],
       [0.25728043, 0.64233345, 0.82027623, ..., 0.64278171, 0.2654959 ,
        0.0150619 ],
       [0.10663021, 0.7674752 , 0.25751682, ..., 0.47690235, 0.86420875,
        0.03329501],
       ...,
       [0.55190602, 0.89107637, 0.35938802, ..., 0.42114102, 0.97121667,
        0.0414624 ],
       [0.87873069, 0.15128475, 0.17590508, ..., 0.59751187, 0.0941181 ,
        0.19304587],
       [0.57807521, 0.51380945, 0.41897232, ..., 0.5599062 , 0.08064351,
        0.71741084]])

In [24]:
a = torch.rand(1200,25)
a

tensor([[0.7802, 0.1532, 0.9607,  ..., 0.9290, 0.1604, 0.8680],
        [0.5478, 0.3615, 0.6994,  ..., 0.2662, 0.9042, 0.7127],
        [0.6768, 0.7561, 0.4156,  ..., 0.0214, 0.0238, 0.7925],
        ...,
        [0.7368, 0.7269, 0.0720,  ..., 0.7282, 0.0201, 0.3742],
        [0.0402, 0.8435, 0.0113,  ..., 0.5344, 0.3073, 0.8463],
        [0.0196, 0.1232, 0.8481,  ..., 0.1411, 0.9525, 0.1716]])